In [1]:
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pprint
from deap import base, creator, tools, algorithms
import pandas as pd
from modules.knapsack_tools import ITEMS_KNAPSACK, MAX_CAPACITY, apply_tournament, KnapsackProblem, run_knapsack_tournament



In [2]:
df_full_knapsack = pd.read_csv('reports/knapsack_full_experiments.csv')
df_full_knapsack.head()

,Unnamed: 0,max_found_at_generation,end_mean_fitness_population,end_max_fitness,initial_population,max_generation,crossover_probability,mutation_probability,selection_method,selection_strategy_name,k_value
0,0,48,961.9,970.0,50,50,0.1,0.1,<function selTournament at 0x00000213A144F520>,Tournament,1
1,1,13,965.9,970.0,50,50,0.1,0.1,<function selTournament at 0x00000213A144F520>,Tournament,2
2,2,4,858.9,860.0,50,50,0.1,0.1,<function selTournament at 0x00000213A144F520>,Tournament,3
3,3,42,935.5,950.0,50,50,0.1,0.1,<function selTournament at 0x00000213A144F520>,Tournament,4
4,4,41,985.2,1000.0,50,50,0.1,0.2,<function selTournament at 0x00000213A144F520>,Tournament,1


In [3]:
# Finding if any, the difference between computing using Torunament or Roulette
df_full_knapsack.columns

Index(['Unnamed: 0', 'max_found_at_generation', 'end_mean_fitness_population',
       'end_max_fitness', 'initial_population', 'max_generation',
       'crossover_probability', 'mutation_probability', 'selection_method',
       'selection_strategy_name', 'k_value'],
      dtype='object')

In [4]:
df_full_knapsack.describe()

,Unnamed: 0,max_found_at_generation,end_mean_fitness_population,end_max_fitness,initial_population,max_generation,crossover_probability,mutation_probability,k_value
count,1728.000000,1728.000000,1728.000000,1728.000000,1728.000000,1728.000000,1728.000000,1728.000000,1728.000000
mean,863.500000,25.643519,1002.413180,1019.622685,300.000000,100.000000,0.250000,0.250000,2.500000
std,498.974949,28.491106,22.614099,22.648084,204.183235,40.836647,0.111836,0.111836,1.118358
min,0.000000,1.000000,858.900000,860.000000,50.000000,50.000000,0.100000,0.100000,1.000000
25%,431.750000,9.000000,998.042500,1030.000000,50.000000,50.000000,0.175000,0.175000,1.750000
50%,863.500000,13.000000,1008.349091,1030.000000,300.000000,100.000000,0.250000,0.250000,2.500000
75%,1295.250000,31.000000,1016.942273,1030.000000,550.000000,150.000000,0.325000,0.325000,3.250000
max,1727.000000,149.000000,1030.000000,1030.000000,550.000000,150.000000,0.400000,0.400000,4.000000


In [13]:
df_full_knapsack.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   1728 non-null   int64  
 1   max_found_at_generation      1728 non-null   int64  
 2   end_mean_fitness_population  1728 non-null   float64
 3   end_max_fitness              1728 non-null   float64
 4   initial_population           1728 non-null   int64  
 5   max_generation               1728 non-null   int64  
 6   crossover_probability        1728 non-null   float64
 7   mutation_probability         1728 non-null   float64
 8   selection_method             1728 non-null   object 
 9   selection_strategy_name      1728 non-null   object 
 10  k_value                      1728 non-null   int64  
dtypes: float64(4), int64(5), object(2)
memory usage: 148.6+ KB


In [4]:
df_full_knapsack['end_max_fitness']

0        970.0
1        970.0
2        860.0
3        950.0
4       1000.0
         ...  
1723    1030.0
1724    1030.0
1725    1030.0
1726    1030.0
1727    1030.0
Name: end_max_fitness, Length: 1728, dtype: float64

In [23]:
df_full_knapsack['max_found_at_generation']

0       48
1       13
2        4
3       42
4       41
        ..
1723     8
1724    10
1725     8
1726     6
1727     5
Name: max_found_at_generation, Length: 1728, dtype: int64

In [9]:
# Correcting column name references and using 'hue' for color coding

# sns.scatterplot(data=df_full_knapsack, 
#                 x='max_found_at_generation', 
#                 y='end_max_fitness')


sns.scatterplot(
                x=[1, 2, 3], 
                y=[1, 2, 3]
                )

<Axes: xlabel='end_max_fitness', ylabel='max_found_at_generation'>

In [29]:
plt.scatter(list(df_full_knapsack['end_max_fitness'])[:10], list(df_full_knapsack['max_found_at_generation'])[:10])
plt.xlabel('end_max_fitness')
plt.ylabel('max_found_at_generation')
plt.title('end_max_fitness vs max_found_at_generation')
plt.show()

C:\Users\wangn\AppData\Local\Temp\ipykernel_36548\538925895.py:5: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
